# Preparation

Preparation biasa dilakukan untuk mempersiapkan data sebelum masuk dalam tahap pemodelan. <br>
Berikut adalah tahapan yang akan dilalui pada data `SC_HW1_bank_data.csv` sebelum tahap pemodelan :
1. Import Library
2. Input Dataset
3. Preprocessing
4. Train-Test Split

## Import Library

In [ ]:
import pandas as pd
import numpy as np

## Input Dataset

In [ ]:
#Membaca data dan memasukkannya ke dalam bentuk Pandas Dataframe
df = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')
df.head()

,RowNumber,CustomerId,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#Jalankan code untuk mengecek nama kolom yang tersedia
df.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
#Hilangkan kolom yang dirasa tidak relevan dengan model (contoh: ID). None dapat diisi dengan nama-nama kolom yang akan digunakan.
#Contoh df = df[['X1','X2', 'Y']].copy()

#START CODE
df = df[['Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']].copy()

## Preprocessing

In [ ]:
#Lakukan One-Hot Encoder untuk data categorical, dengan fungsi pandas get_dummies

#START CODE
categorical_cols = ['Gender', 'Geography']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(df.head())

   Age  Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0   42       2       0.00              1          1               1   
1   41       1   83807.86              1          0               1   
2   42       8  159660.80              3          1               0   
3   39       1       0.00              2          0               0   
4   43       2  125510.82              1          1               1   

   EstimatedSalary  Exited  Gender_Male  Geography_Germany  Geography_Spain  
0        101348.88       1            0                  0                0  
1        112542.58       0            0                  0                1  
2        113931.57       1            0                  0                0  
3         93826.63       0            0                  0                0  
4         79084.10       0            0                  0                1  


In [ ]:
#Pisahkan mana X (feature) dengan Y,
#Y adalah kolom "Exited"

#START CODE
X = df.drop('Exited', axis=1)
y = df['Exited']

In [ ]:
#Lakukan Scaler dan/atau Normalisasi jika diperlukan
from sklearn.preprocessing import MinMaxScaler
#START CODE
scaler = MinMaxScaler()
X_transform = scaler.fit_transform(X)

In [ ]:
#Ini digunakan jika dilakukan scaler/Normalisas. Jika tidak, code ini bisa dilewat dan diganti dengan code yang ada di dalam komen
X_transform = pd.DataFrame(X_transform, columns = X.columns)
#X_transform = X.copy()

## Train-Test Split

In [ ]:
#Split menjadi train dan test dengan test_size 25%
#Tidak perlu mengubah code ini

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_transform,y,test_size = 0.25,random_state = 123)

# Modeling

## Model1


Random Forest adalah sebuah algoritma machine learning yang digunakan untuk tugas klasifikasi dan regresi. Algoritma ini bekerja dengan cara membangun sejumlah besar pohon keputusan (decision trees) yang bekerja secara bersamaan.

In [ ]:
from numpy.random.mtrand import RandomState
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}

from sklearn.ensemble import RandomForestClassifier
#START CODE
model1 = RandomForestClassifier(random_state=42)
params = {
    'n_estimators': [64, 100],
    'max_features': [None, 'auto', 'sqrt', 'log2'],
    'max_depth': [10],
    'min_samples_split': [10],
    'min_samples_leaf': [2],
    'bootstrap': [True],
    'oob_score': [True]
    }
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model1,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 100,
 'oob_score': True}

In [ ]:
#lakukan evaluasi dengan beberapa maetric di bawah ini
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score:")
print(100 * (accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1983
           1       0.78      0.45      0.57       517

    accuracy                           0.86      2500
   macro avg       0.83      0.71      0.74      2500
weighted avg       0.85      0.86      0.84      2500

Confusion Matrix:
[[1917   66]
 [ 284  233]]
Accuracy Score:
86.0


## Model2
K-Nearest Neighbors (KNN) adalah algoritma machine learning yang digunakan untuk tugas klasifikasi dan regresi. Dalam KNN, prediksi untuk suatu data baru didasarkan pada mayoritas label dari k-nearest data tetangga dalam ruang fitur. Artinya, jika suatu data memiliki tetangga terdekatnya yang sebagian besar termasuk ke dalam kelas tertentu, maka data tersebut juga akan diklasifikasikan ke dalam kelas yang sama.

In [ ]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}

from sklearn.neighbors import KNeighborsClassifier
#START CODE
model3 = KNeighborsClassifier()
params = {
          'n_neighbors': [15, 20, 30, 45, 64],
          'weights': ['uniform', 'distance'],
          'metric': ['euclidean', 'manhattan'],
          'p': [1, 2],
          'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
          'leaf_size': [10, 20, 30]
         }
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model3,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'algorithm': 'auto',
 'leaf_size': 10,
 'metric': 'manhattan',
 'n_neighbors': 15,
 'p': 1,
 'weights': 'distance'}

In [ ]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score:")
print(100 * (accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90      1983
           1       0.68      0.28      0.40       517

    accuracy                           0.82      2500
   macro avg       0.76      0.62      0.65      2500
weighted avg       0.80      0.82      0.79      2500

Confusion Matrix:
[[1915   68]
 [ 372  145]]
Accuracy Score:
82.39999999999999


## Model3
Gradient Boosting Classifier adalah algoritma machine learning yang digunakan untuk tugas klasifikasi. Algoritma ini bekerja dengan cara membangun serangkaian model prediksi (biasanya pohon keputusan) secara berurutan, di mana setiap model berusaha untuk memperbaiki kesalahan prediksi model sebelumnya.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#START CODE
model3 = GradientBoostingClassifier()
params = {
          'n_estimators': [100, 200, 300],  # Number of boosting stages to be used
          'max_depth': [3, 4, 5],  # Maximum depth of individual trees
          'max_features': ['auto', 'sqrt', 'log2', None]
         }
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model3,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100}

In [ ]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score:")
print(100 * (accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1983
           1       0.77      0.51      0.61       517

    accuracy                           0.87      2500
   macro avg       0.83      0.74      0.77      2500
weighted avg       0.86      0.87      0.86      2500

Confusion Matrix:
[[1903   80]
 [ 252  265]]
Accuracy Score:
86.72


###Model dengan akurasi yang terbaik adalah Gradient Boosting Classifier karena mendapatkan nilai akurasi setidaknya 86.72 (> 86) dibandingkan dua model lainnya yaitu KNN (82.39) dan Random Forest (86.0)